# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gecko Multimodal Embeddings - Korean text embedding test
* This notebook explains how to use Korean embeddings and understand vectorization.
* Refer to the link for more information about the embeddings.
 * https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings
 * https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.7 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


# Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [3]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [4]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

import vertexai
from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

## Functions to evaluate similarity

## Multimodal embedding

* https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings

In [24]:
import vertexai
from vertexai.vision_models import Image, MultiModalEmbeddingModel

def get_gecko_embeddings(image_path, contextual_text):

  model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
  image = Image.load_from_file(image_path)

  embeddings = model.get_embeddings(
      image=image,
      contextual_text=contextual_text,
      dimension=1408, # dimension must be one of 128, 256, 512, 1408.

  )
  print(f"Image Embedding: {embeddings.image_embedding}")
  print(f"Text Embedding: {embeddings.text_embedding}")

  return embeddings


In [25]:
import numpy as np
import torch
import torch.nn as nn

def cal_dot_product(a, b):
  """ torch dot product similarity """

  return torch.dot(a, b)

def cal_cosine(a, b):
  """ torch cosine similarity """

  cos = nn.CosineSimilarity(dim=0, eps=1e-6)
  output = cos(a, b)
  return output

def cal_mm(a, b):
  """ torch matrix multiplication """

  if len(a.shape) == 1: a = a.unsqueeze(0)
  if len(b.shape) == 1: b = b.unsqueeze(0)

  a_norm = a / a.norm(dim=1)[:, None]
  b_norm = b / b.norm(dim=1)[:, None]
  return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [23]:
from google.colab import files
from IPython.display import Image
uploaded = files.upload()

In [26]:
input = get_gecko_embeddings("./cha.jpg", "차승원" )

cha_1 = get_gecko_embeddings("./cha1.jpg", "차승원" )
cha_2 = get_gecko_embeddings("./cha2.png", "차승원" )
cha_3 = get_gecko_embeddings("./cha3.jpeg", "차승원" )
cha_4 = get_gecko_embeddings("./cha4.jpg", "차승원" )
cha_5 = get_gecko_embeddings("./cha5.jpg", "차승원" )
cha_6 = get_gecko_embeddings("./cha6.jpg", "차승원" )
cha_7 = get_gecko_embeddings("./cha7.jpg", "차승원" )


Image Embedding: [-0.0312805399, 0.0574740581, 0.0213833619, 0.0198861659, -0.0427943096, 0.0803418383, 0.0678316876, -0.0610869154, -0.000465603662, 0.0115481084, 0.00538443122, -0.0515602827, -0.0239016376, 0.0203566309, -0.00204478018, 0.0122857532, 0.0275406539, 0.0610668585, -0.0804291889, 0.00612252, -0.0378022119, -0.02822336, 0.0110037765, -0.0168183502, 0.0442803465, 0.0417933576, -0.000835351, -0.036176119, 0.0105781462, 0.0182711668, -0.0601209328, -0.0251248349, 0.0553361215, -0.0452847108, 0.0170305017, -0.0662188381, 0.00585787417, -0.0525446385, 0.0287176613, -0.00538886106, 0.0198858492, 0.000827914278, -0.038951844, -0.00268634921, -0.0378017128, -0.0130645214, -0.0451603457, -0.0169339329, 0.0362262763, -0.0204349887, -0.000370297203, -0.0311287884, -0.00963027775, -0.00965016149, -0.0278945509, -0.0213743, -0.0184036233, 0.0103366217, 0.0301500894, -0.0457974225, -3.41153536e-05, 0.0321984254, 0.0132157262, 0.0126398476, 0.0179527383, 0.0864712372, -0.0164208207, 0.0

In [27]:
len(input.image_embedding)

512

In [28]:
others_1 = get_gecko_embeddings("./gd.jpg", "권지용" )
others_2 = get_gecko_embeddings("./um.jpg", "엄태구" )
others_3 = get_gecko_embeddings("./lee.png", "이경규" )

Image Embedding: [-0.0317396894, 0.0111037316, -0.026860429, 0.0485707149, 0.0106458068, 0.0570930839, 0.0203890838, -0.00658138143, -0.00458873715, -0.0690460727, 0.00590008684, 0.000850672135, 0.00358811044, 0.0365189761, 0.0408734567, -0.0317193307, 0.00467396062, 0.0393425412, -0.0436252095, -0.00922688656, 0.0514194556, -0.0324287787, 0.0102369254, 0.00809582509, 0.0251961146, 0.0434884392, 0.00798794627, -0.00694933254, 0.0479490422, 0.031256523, 0.00689466111, -0.0426698588, 0.0150912469, -0.0349973328, 0.0572912209, 0.0427811332, 0.023762811, -0.0393959954, 0.0242242701, 0.0101799294, -0.0674337149, -0.0249384493, -0.00668245368, -0.0498412624, -0.0490525812, 0.00243921578, 0.00406718068, -0.0311001278, -0.00465977704, -0.0578086488, 0.00731501, -0.0271722414, -0.0445604734, 0.0279569924, 0.0326983184, 6.68350185e-05, 0.0147777628, -0.00278471923, -0.0492606722, 0.0027926676, -0.0466888286, 0.00503244204, -0.00677618571, 0.0158162415, 0.00112631416, 0.0385611579, -0.012887164, 

In [29]:
input_embed = torch.Tensor(input.image_embedding)

cha_embed1 = torch.Tensor(cha_1.image_embedding)
cha_embed2 = torch.Tensor(cha_2.image_embedding)
cha_embed3 = torch.Tensor(cha_3.image_embedding)
cha_embed4 = torch.Tensor(cha_4.image_embedding)
cha_embed5 = torch.Tensor(cha_5.image_embedding)
cha_embed6 = torch.Tensor(cha_6.image_embedding)
cha_embed7 = torch.Tensor(cha_7.image_embedding)

print(f"cha_1 : {cal_dot_product(input_embed,cha_embed1 )}")
print(f"cha_2 : {cal_dot_product(input_embed,cha_embed2 )}")
print(f"cha_3 : {cal_dot_product(input_embed,cha_embed3 )}")
print(f"cha_4 : {cal_dot_product(input_embed,cha_embed4 )}")
print(f"cha_5 : {cal_dot_product(input_embed,cha_embed5 )}")
print(f"cha_6 : {cal_dot_product(input_embed,cha_embed6 )}")
print(f"cha_7 : {cal_dot_product(input_embed,cha_embed7 )}")

# -------
others_embed1 = torch.Tensor(others_1.image_embedding)
others_embed2 = torch.Tensor(others_2.image_embedding)
others_embed3 = torch.Tensor(others_3.image_embedding)

print("-"*20)

print(f"others_1 : {cal_dot_product(input_embed,others_embed1 )}")
print(f"others_2 : {cal_dot_product(input_embed,others_embed2 )}")
print(f"others_3 : {cal_dot_product(input_embed,others_embed3 )}")


cha_1 : 0.5506726503372192
cha_2 : 0.667943000793457
cha_3 : 0.5904831886291504
cha_4 : 0.6720057129859924
cha_5 : 0.7266963720321655
cha_6 : 0.6113201379776001
cha_7 : 0.6500333547592163
--------------------
others_1 : 0.54859858751297
others_2 : 0.6527541875839233
others_3 : 0.6431349515914917
